In [1]:
import pandas as pd
import numpy as np
import sqlite3 as db
import os

In [2]:
%pwd

path = os.getcwd()

# Utils

### Import des data

In [3]:
def import_txt():
    routes_df=pd.read_csv('./SEM-GTFS/routes.txt')
    stops_df=pd.read_csv('./SEM-GTFS/stops.txt')
    stop_times_df=pd.read_csv('./SEM-GTFS/stop_times.txt')
    agency_df=pd.read_csv('./SEM-GTFS/agency.txt')
    trips_df=pd.read_csv('./SEM-GTFS/trips.txt')
    return routes_df, stops_df, stop_times_df, agency_df, trips_df

In [4]:
routes_df, stops_df, stop_times_df, agency_df, trips_df = import_txt()

### Get infos

In [5]:
def get_infos(db_file):
    conn = db.connect(db_file)
    cur = conn.cursor()
    
    # Retrieve the names of all tables in the database
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [table[0] for table in cur.fetchall()]
    
    # Retrieve the columns in each table
    columns = {}
    for table in tables:
        cur.execute("PRAGMA table_info({})".format(table))
        columns[table] = [column[1] for column in cur.fetchall()]
    
    cur.close()
    conn.close()
    print("°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°")
    print("------------------------------ TABLES --------------------------------")
    print("")
    print("Tables:", tables)
    print("")
    print("------------------------------ COLUMNS -------------------------------")
    for table in tables:
        print("")
        print(f"----> {table} <----")
        print("Columns --> ", columns[table])
    print("°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°")

### Query the DB

In [6]:
# The name has to be in the same folder as the notebook
def query_db(name, query):
    
    conn = db.connect(name)
    cur = conn.cursor()
    cur.execute(query)
    column_names = [description[0] for description in cur.description]
    results = cur.fetchall()
    cur.close()
    conn.close()
    
    #df_results = pd.DataFrame(results, columns=column_names)

    return df_results

### Generate the empty table

In [7]:
def create_database(database_file):
    # Remove the database file if it already exists
    if os.path.exists(database_file):
        os.remove(database_file)

    # Connect to the database and create the tables
    conn = db.connect(database_file)
    c = conn.cursor()
    c.execute('''CREATE TABLE agency (
                    agency_id TEXT PRIMARY KEY,
                    agency_name TEXT,
                    agency_url TEXT,
                    agency_timezone TEXT,
                    agency_lang TEXT,
                    agency_phone TEXT,
                    agency_fare_url TEXT,
                    agency_email TEXT)''')
    c.execute('''CREATE TABLE routes (
                    route_id TEXT PRIMARY KEY,
                    agency_id TEXT,
                    route_short_name TEXT,
                    route_long_name TEXT,
                    route_description TEXT,
                    route_type TEXT,
                    route_url TEXT,
                    route_color TEXT,
                    route_text_color TEXT,
                    route_sort_order TEXT,
                    FOREIGN KEY (agency_id) REFERENCES agency(agency_id))''')
    c.execute('''CREATE TABLE trips (
                    route_id TEXT,
                    service_id TEXT,
                    trip_id TEXT PRIMARY KEY,
                    trip_headsign TEXT,
                    trip_short_name TEXT,
                    direction_id INT,
                    block_id TEXT,
                    shape_id TEXT,
                    wheelchair_accessible INT,
                    bikes_allowed INT,
                    FOREIGN KEY (route_id) REFERENCES routes(route_id))''')
    c.execute('''CREATE TABLE stop_times (
                    trip_id TEXT,
                    arrival_time TEXT,
                    departure_time TEXT,
                    stop_id TEXT,
                    stop_sequence INT,
                    stop_headsign TEXT,
                    pickup_type INT,
                    drop_off_type INT,
                    continuous_pickup INT,
                    continuous_drop_off INT,
                    shape_dist_traveled FLOAT,
                    timepoint INT,
                    PRIMARY KEY (trip_id, arrival_time, departure_time, stop_id),
                    FOREIGN KEY (trip_id) REFERENCES trips(trip_id),
                    FOREIGN KEY (stop_id) REFERENCES stops(stop_id))''')
    c.execute('''CREATE TABLE stops (
                    stop_id TEXT PRIMARY KEY,
                    stop_code TEXT,
                    stop_name TEXT,
                    stop_desc TEXT,
                    stop_lat TEXT,
                    stop_lon TEXT,
                    zone_id TEXT,
                    stop_url TEXT,
                    location_type INT,
                    parent_station TEXT,
                    stop_timezone TEXT,
                    wheelchair_boarding INT,
                    level_id TEXT,
                    platform_code TEXT)''')

    # Save the changes and close the connection
    conn.commit()
    conn.close()


In [8]:
create_database("test.db")

# I - Implémenter une fonction qui génère une commande insertion SQL

In [9]:
db_name = "gtfs_tag.db"

In [10]:
get_infos(db_name)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
------------------------------ TABLES --------------------------------

Tables: ['agency', 'routes', 'trips', 'stop_times', 'stops']

------------------------------ COLUMNS -------------------------------

----> agency <----
Columns -->  ['agency_id', 'agency_name', 'agency_url', 'agency_timezone', 'agency_lang', 'agency_phone', 'agency_fare_url', 'agency_email']

----> routes <----
Columns -->  ['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_description', 'route_type', 'route_url', 'route_color', 'route_text_color', 'route_sort_order']

----> trips <----
Columns -->  ['route_id', 'service_id', 'trip_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed']

----> stop_times <----
Columns -->  ['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'stop_headsig

### Generate dict of data to insert

In [11]:
#Transformation des fichiers en dictionnaires
routes_dic=routes_df.to_dict(orient='dict')
stops_dic=stops_df.to_dict(orient='dict')
stop_times_dic=stop_times_df.to_dict(orient='dict')
agency_dic=agency_df.to_dict(orient='dict')
trips_dic=trips_df.to_dict(orient='dict')
agency_dic

{'agency_id': {0: 'SEM'},
 'agency_name': {0: 'Mobilités M - Tag'},
 'agency_url': {0: 'https://www.mobilites-m.fr/'},
 'agency_timezone': {0: 'Europe/Paris'},
 'agency_lang': {0: 'FR'},
 'agency_phone': {0: 438703870}}

### Créer une query générale INSERT table_name  ...COLUMNS... puis toutes les VALUE

In [12]:
table_name = "agency"
dict_name = agency_dic

In [13]:
def generate_insertion_query(table_name: str, a_dict: dict) -> str:
    columns = ", ".join(a_dict.keys())
    values = ", ".join("'" + str(x).replace("'", "''") + "'" for x in a_dict.values())
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
    return query

In [14]:
query = generate_insertion_query(table_name, dict_name)

In [15]:
def save_sql_query_to_file(query, file_path):
    with open(file_path, 'w') as file:
        file.write(query)
    print(f'Saved SQL query to {file_path}')

In [16]:
file = f"{path}/{table_name}_query_alone.txt"
save_sql_query_to_file(query, file)

Saved SQL query to /Users/manulabricole/Documents/CDN/BDD/agency_query_alone.txt


### Maintenant que Agency est ajouté, en principe plus d'erreur de foreign key. Test ajout routes

In [17]:
table_name = "routes"
dict_name = routes_dic

In [18]:
query = generate_insertion_query(table_name, dict_name)
file = f"{path}/{table_name}_query_alone.txt"
save_sql_query_to_file(query, file)

Saved SQL query to /Users/manulabricole/Documents/CDN/BDD/routes_query_alone.txt


# II - Implémenter une fonction qui génère des commandes d’insertion SQL

In [19]:
line=routes_df.iloc[4].dropna()
line

agency_id                                                         SEM
route_id                                                            5
route_short_name                                                   C5
route_long_name     GRENOBLE Palais de Justice - Gare / GIÈRES Uni...
route_type                                                          3
route_color                                                    F5D24D
route_text_color                                               000000
Name: 4, dtype: object

In [20]:
def dataframe_to_query(table_name, df):
    for i in range(len(df)):
        line=df.iloc[i].dropna()
        query= "INSERT INTO {} {} VALUES {};\n ".format(table_name, tuple(line.index), tuple(line.to_list()))
    return query

In [21]:
query_df = dataframe_to_query("routes", routes_df)

### III - implémenter une procédure qui crée un fichier SQL

In [22]:
def gen_insert_file(filename,table_name,df):
    with open(filename,'w') as file:
        file.write("BEGIN;")
        for i in range(len(df)):
            line=df.iloc[i].dropna()
            query= "INSERT INTO {} {} VALUES {};\n ".format(table_name, tuple(line.index), tuple(line.to_list()))
            file.write(query)
        file.write("COMMIT;")
    return filename

In [23]:
gen_insert_file("./routes.sql","routes", routes_df)
gen_insert_file("./stops.sql","stops", stops_df)
gen_insert_file("./stop_times.sql","stop_times", stop_times_df)
gen_insert_file("./agency.sql","agency", agency_df)
gen_insert_file("./trips.sql","trips", trips_df)

'./trips.sql'

# Methode Pandas

In [24]:
pandas_db = "pandas_methode.db"

In [25]:
create_database(pandas_db)
routes_df, stops_df, stop_times_df, agency_df, trips_df = import_txt()

DB=db.connect(pandas_db)
connecteur=db.connect(pandas_db)

routes_df.to_sql(name='routes',con=connecteur, if_exists='replace') 
stops_df.to_sql(name='stops',con=connecteur, if_exists='replace')
stop_times_df.to_sql(name='stops_times',con=connecteur, if_exists='replace')
agency_df.to_sql(name='agency',con=connecteur, if_exists='replace')
trips_df.to_sql(name='trips',con=connecteur, if_exists='replace')

DB.close()